In [38]:
import csv
import numpy
import numpy as np
import scipy as sp
import scipy.stats
import hashlib
import matplotlib.pyplot as plt

FUNZIONI UTILI PER L'ANALISI DEL TESTO

In [39]:
# crea un dizionario, in cui ad ogni parola (chiave) corrisponde una lista
# di babel synset presi del file "SemEval17_IT_senses2sysensts.txt"
def get_senses_dictionary(word_list):
    senses_for_words = dict()
    file = open("SemEval17_IT_senses2synsets.txt", "r", encoding="utf8")
    lines = file.readlines()
    for i in range(0, len(lines)):
        line = lines[i]
        line = line.replace("\n", "").replace("#", "")
        if line in word_list:
            while True:
                i += 1
                babel_synset = lines[i].replace("\n", "")
                if "#" in babel_synset:  # vuol dire che non sto considerando più un babel synset
                    break
                if line not in senses_for_words:
                    senses_for_words[line] = [babel_synset]
                else:
                    senses_for_words[line].append(babel_synset)
    return senses_for_words

# crea un dizionario delle annotazioni, in cui ad ogni coppia di parole
# prese dal file delle annotazioni "annotations1.tsv" associa il valore
# di similarità annotato da un essere umano
def get_human_similarities_dictionary():
    human_similarities = dict()
    annotations_file = open("annotations1.tsv")
    read_tsv = csv.reader(annotations_file, delimiter="\t")
    for row in read_tsv:
        if row[0]: #verifico che non sia una riga vuota (può capitare)
            human_similarities[(row[0],row[1])] = row[2]
    annotations_file.close()
    return human_similarities

# crea un dizionario delle annotazioni, in cui ad ogni coppia di parola
# prese dal file delle annotazioni "annotations2.tsv" associa una coppia
# di BABEL synset annotati da un essere umano sulla base delle annotazioni
# di similarità del file "annotations1.tsv"
def get_human_synsets_dictionary():
    human_synsets = dict()
    annotations_file = open("annotations2.tsv")
    read_tsv = csv.reader(annotations_file, delimiter="\t")
    for row in read_tsv:
        if row[0]: #verifico che non sia una riga vuota (può capitare)
            human_synsets[((row[0],row[1]))] = (row[2],row[3])
    annotations_file.close()
    return human_synsets

# restituisce tutte le parole presenti nella coppie valutate nel dizionario
# delle annotazioni umane che viene dato in input
def get_word_list(human_similarities_dictionary):
    word_list = []
    for pair in human_similarities_dictionary.keys():
        word_list.append(pair[0])
        word_list.append(pair[1])
    return word_list

# crea una dizionario che associa ad ogni coppia di parole del dizionario
# delle annotazioni umane delle similarità, un valore di similarità dato dalla massimizzazione della similarità
# del coseno tra l'insieme dei vettori nasari associati ad una parola
# e l'insieme dei vettori nasari associati all'altra parola
# CONSEGNA 1
def get_NASARI_similarities_dictionary(human_similarities_dictionary, senses_dictionary):
    similarity_dictionary = dict()
    for word_pair in human_similarities_dictionary:

        try:
            word1_senses = senses_dictionary[word_pair[0]]
            word2_senses = senses_dictionary[word_pair[1]]

            word1_vectors = get_NASARI_vectors(word1_senses)
            word2_vectors = get_NASARI_vectors(word2_senses)

            similarity_value = max_cosine_similarity(word1_vectors, word2_vectors)[0]
            similarity_dictionary[word_pair] = similarity_value

        except KeyError:  # una delle due parole della coppia non c'è nel file "SemEval17_IT_senses2synsets.txt"
            print("La coppia ", word_pair, "non e' stata valutata")
    return similarity_dictionary

# crea un dizionario che associa ad ogni coppia di parole del dizionario
# delle annotazioni umane dei sensi, una coppia di babel synset che massimizza la similarità
# del coseno tra l'insieme dei vettori associati alla prima parola
# e l'insieme dei vettori associati alla seconda parola
# CONSEGNA 2
def get_word_pair_synset_pair_dictionary(human_synsets_dictionary, senses_dictionary):
    synsets_dictionary = dict()
    for word_pair in human_synsets_dictionary:
        try:
            word1_senses = senses_dictionary[word_pair[0]]
            word2_senses = senses_dictionary[word_pair[1]]

            word1_vectors = get_NASARI_vectors(word1_senses)
            word2_vectors = get_NASARI_vectors(word2_senses)

            # TO-DO
            synset_pair = max_cosine_similarity(word1_vectors, word2_vectors)[1]
            synsets_dictionary[word_pair] = synset_pair

        except KeyError:  # una delle due parole della coppia non c'è nel file "SemEval17_IT_senses2synsets.txt"
            print("La coppia ", word_pair, "non e' stata valutata")
    return synsets_dictionary

# cerca un vettore NASARI per ogni babel synset in input
# associati ad una parola. Resituisce un dizionario
# che avrà come chiavi i babel synset e come valori
# i vettori NASARI associati
def get_NASARI_vectors(word_senses):
    word_vectors = dict()

    NASARI_file = open("mini_NASARI.tsv", encoding="utf8")
    read_tsv = csv.reader(NASARI_file, delimiter="\t")

    for row in read_tsv:
        babel_synset = row[0].split("__")[0]
        if babel_synset in word_senses:
            vector = [float(val) for val in row[1:]]
            word_vectors[babel_synset] = vector

    NASARI_file.close()
    return word_vectors

# massimizza la cosine_similarity tra due liste di vettori distribuzionali
def max_cosine_similarity(word_vector1, word_vectors2):
    max_cos_similarity = 0
    for babel_synset1 in word_vector1.keys():
        for babel_synset2 in word_vectors2.keys():
            cos_similarity = c_similarity(word_vector1[babel_synset1], word_vectors2[babel_synset2])
            if cos_similarity > max_cos_similarity:
                max_cos_similarity = cos_similarity
                synset_pair = (babel_synset1, babel_synset2)
    return max_cos_similarity, synset_pair

# calcola la similarità del coseno tra due vettori numerici
# restituisce quindi il rapporto tra il prodotto scalare dei due vettori e il
# prodotto della loro norma
def c_similarity(vect1, vect2):
    numerator = numpy.dot(vect1, vect2)
    denominator = numpy.linalg.norm(vect1) * numpy.linalg.norm(vect2)
    return numerator / denominator

Mappa un cognome su uno dei 10 insiemi di coppie da annotare

In [40]:
def get_range(surname):
    nof_elements = 500
    base_idx = (abs(int(hashlib.sha512(surname.encode('utf-8')).hexdigest(), 16)) % 10)
    idx_intervallo = base_idx * 50+1
    return idx_intervallo 

input_name = "Parisi"

values = []
sx = get_range(input_name)
values.append(sx)
dx = sx+50-1
intervallo = "" + str(sx) + "-" + str(dx)
print('{:15}:\tcoppie nell\'intervallo {}'.format(input_name, intervallo))

Parisi         :	coppie nell'intervallo 451-500


Processo della prima annotazione manuale dell'utente

Vengono esaminate le coppie di termini dal file "it.test.data.txt" in base al cognone (Parisi: coppie nell'intervallo 451-500) che vanno da INTERVAL_START a INTERVAL_END

Il processo di annotazione è fatto in modo manuale dall'utente che assegna un punteggio di similarità da 0 a 4.

0: Totally dissimilar and unrelated

1: Dissimilar

2: Slightly similar

3: Similar

4: Very similar


In [41]:
INTERVAL_START = 451
INTERVAL_END = 500

with open('annotations1.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')

    with open("it.test.data.txt", "r", encoding="utf8") as fp:
        for i, line in enumerate(fp):
            if i >= INTERVAL_START - 1:
                line = line.replace("\n", "").split("\t")
                print("==============================")
                word1, word2 = line[0], line[1]
                print(word1, " ", word2)
                similarity_value = float(input("Inserire similaritÃ (0-4): "))
                similarity_value = format(similarity_value, '.1f')
                tsv_writer.writerow([word1, word2, similarity_value])
                print("==============================")
            if i == INTERVAL_END - 1:
                break
    fp.close()
    out_file.close()

recessione   PIL
Cesare   Giulio Cesare
paziente   sessione
comportamentismo   terapia
imperatore   costituzione
matematico   spettacolo
entropia   informazione
acqua di rose   olio di rosa
agrume   pompelmo
Regina Vittoria   Inghilterra
Giochi Olimpici   spirito
vescovo   musulmano
uomo   sospetto
meteorite   Terra
simbolo   segno
antropologia   New York
tramonto   tavolo
cittadina   città
giacca   acqua minerale
natura   flora
subroutine   compilatore
Hamadan   Roma
ombrello   stufa
onore   stima
insegna   dignità
KFC   McDonald's
joystick   radar
basmati   riso jasmine
medaglia   scarpe da ginnastica
legge   piscina
sorgente   scatola
teatro   batteria
flora   web browser
camicia   cardigan
poema   ritmo
profeta   prete
Oscar   stadio
backgammon   Go
farfalla   rosa
recinto   salto
nichilismo   film
asteroide   stella
sommossa   disegno
intimo   corpo
Boeing   aereo
cameo   interpretazione
semestre   quadrimestre
arancia   agrume
ghiacciaio   riscaldamento globale
galleria   percors

Processo della seconda annotazione manuale dell'utente

Anche in questo in caso vengono prese in considerazione le coppie di parole presenti nel file in base al cognome.

Il processo di annotazione consiste nel predere la coppia di termini associarla alla coppia di bable synset ID e cosiderare, per entrambe i termini, i 3 sensi più simili presenti su WordNet, in questo ordine: Term1 Term2 BS1 BS2 Terms_in_BS1 Terms_in_BS2

In [42]:
with open('annotations2.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')

    tsv_writer.writerow(['recessione', 'PIL', 'bn:00066516n','bn:00037570n','condizione macroeconomica,fattori produttivi,crescita economica','pil, P.I.L.,prodotto interno lordo'])
    tsv_writer.writerow(['Cesare', 'Giulio Cesare', 'bn:00014550n','bn:00014550n','Gaio Giulio Cesare,Caio Giulio Cesare,imperatore ','Gaio Giulio Cesare,Caio Giulio Cesare ,imperatore'])
    tsv_writer.writerow(['paziente', 'sessione', 'bn:00061017n','bn:00076588n',' ammalato,assistito,cliente ','fase,periodo,sessione'])
    tsv_writer.writerow(['comportamentismo', 'terapia', 'bn:00009659n','bn:00076843n','behaviourismo,psicologia,comportamento  ','guarigione,malattie,metodi'])
    tsv_writer.writerow(['imperatore', 'costituzione', 'bn:00014550n','bn:00059480n','Gaio Giulio Cesare,Caio Giulio Cesare,imperatore','organizzazione,formazione,organizzativo'])
    tsv_writer.writerow(['matematico', 'spettacolo', 'bn:00053834n','bn:00067553n','persona,ricerche,sperimentazioni','rivista ,Teatro di rivista,spettacolo '])
    tsv_writer.writerow(['entropia', 'informazione', 'bn:00031061n','bn:00046705n','casualita,azzardo,grandezza','Conoscenza,studio,dati '])
    tsv_writer.writerow(['acqua di rose', 'olio di rosa', 'bn:00068288n','bn:00007010n','giulebbe,bevanda,estratto di rosa ','petali,rosa,estratto '])
    tsv_writer.writerow(['agrume', 'pompelmo', 'bn:00019301n','bn:00019313n','frutti,piante,Rutaceae','Agrume,buccia gialla,amarognolo '])
    tsv_writer.writerow(['Regina Vittoria', 'Inghilterra', 'bn:00065652n','bn:00013173n','regina,Regno Unito,Imperatrice ','Gran Bretagna,Regno Unito,Irlanda del Nord '])
    tsv_writer.writerow(['Giochi Olimpici', 'spirito', 'bn:00058910n','bn:00040370n','olimpiade,Olimpiadi ,Manifestazione','fantasma,spettro,apparizione '])
    tsv_writer.writerow(['vescovo', 'musulmano', 'bn:27267995n','bn:00055975n',' responsabile,chiese,cattolicesimo ','mussulmano,islamico,maomettano'])
    tsv_writer.writerow(['uomo', 'sospetto', 'bn:00044576n','bn:00025884n','essere umano,Homo,umano ','convenuto,accusato ,imputato '])
    tsv_writer.writerow(['meteorite', 'Terra', 'bn:00054602n','bn:00029424n','meteorite,Oggetto,spazio  ','mondo ,globo,terrestre '])
    tsv_writer.writerow(['simbolo', 'segno', 'bn:00075652n','bn:00075652n','segno,significato convenzionale,elemento ','segno,significato convenzionale,elemento'])
    tsv_writer.writerow(['antropologia', 'New York', 'bn:00004584n','bn:00041611n','Scienza,uomo,entita biologica ','Nuova York ,stato di New York,citta'])
    tsv_writer.writerow(['tramonto', 'tavolo', 'bn:05692316n','bn:00075813n','crepuscolo,notte,illuminazione','tavola,mobilio,piano'])
    tsv_writer.writerow(['cittadina', 'citta', 'bn:00070724n','bn:00077773n','Insediamento umano,villaggio,borgo ','citta,cittadina ,paese '])
    tsv_writer.writerow(['giacca', 'acqua minerale', 'bn:00047823n','bn:00055131n','cappotto,casacca,giacchetta ','minerale,bottiglia,acqua sorgiva '])
    tsv_writer.writerow(['natura', 'flora', 'bn:00057017n','bn:00035324n','fenomeni,forze,cose','pianta,Plantae,vegetale '])
    tsv_writer.writerow(['subroutine', 'compilatore','bn:00036826n','bn:00021344n','sottoprogramma,procedura,procedimento','Programma,linguaggio,istruzioni'])
    tsv_writer.writerow(['Hamadan', 'Roma', 'bn:03266645n','bn:00015556n','citta,Iran,Ecbatana','Citta Eterna,Comune di Roma,impero romano '])
    tsv_writer.writerow(['ombrello', 'stufa', 'bn:00078920n','bn:00074479n','parapioggia,paracqua,ombrellone','fornello,fuoco,apparecchio'])
    tsv_writer.writerow(['onore', 'stima', 'bn:00027103n','bn:00031973n','dignita,decoro,onore','valutazione,estimazione,perizia'])
    tsv_writer.writerow(['insegna', 'dignita', 'bn:00034960n','bn:00027103n','bandiera,segnalazioni,identificazione','decoro,onore,reputazione'])
    tsv_writer.writerow(['KFC', 'McDonalds', 'bn:01826071n','bn:01826071n',' catena,ristoranti,fast food','catena,ristoranti,fast food'])
    tsv_writer.writerow(['joystick', 'radar', 'bn:00022301n','bn:00054808n','cloche,barra,comando','radiolocalizzatore,posizione,rilevamento'])
    tsv_writer.writerow(['basmati', 'riso jasmine', 'bn:00567353n','bn:02666084n','riso,grano,fragranza ','riso,chicco lungo,fiori'])
    tsv_writer.writerow(['medaglia', 'scarpe da ginnastica', 'bn:23946490n','bn:00042319n','riconoscimento,Attestazione,distinzione','sneaker,takkies,attivita sportive'])
    tsv_writer.writerow(['legge', 'piscina', 'bn:00048655n','bn:00056911n','diritto,giurisprudenza,legislazione','vasca,piscina coperta,nuotare'])
    tsv_writer.writerow(['sorgente', 'scatola', 'bn:00036077n','bn:00012524n','fonte,polla,scaturigine','cassetta,astuccio,cassa'])
    tsv_writer.writerow(['teatro', 'batteria', 'bn:00045002n','bn:00028891n','Edificio,opere,spettacolo ','tamburo,Membranofoni,membranofono '])
    tsv_writer.writerow(['flora', 'web browser', 'bn:00035324n','bn:00013447n','pianta,Plantae,vegetale','browser,navigatore,informatica'])
    tsv_writer.writerow(['camicia', 'cardigan', 'bn:00071142n','bn:00015958n','Indumento,cotone,maniche','golf,maglione,bottoni'])
    tsv_writer.writerow(['poema', 'ritmo', 'bn:15205441n','bn:00009396n','poesia,versi,narrativo','tempo,pause,intervalli'])
    tsv_writer.writerow(['profeta', 'prete', 'bn:00064759n','bn:00024459n','aedo,divinatore,vate','pastore,curato,Parroco'])
    tsv_writer.writerow(['Oscar', 'stadio', 'bn:00000571n','bn:00005532n','Premio Oscar,Academy Award, cinema','arena,campo sportivo,palazzetto dello sport'])
    tsv_writer.writerow(['backgammon', 'Go', 'bn:00007779n','bn:00040833n','tavola reale,sbaraglino,tric-trac','gioco da tavolo,giocatori ,Cina'])
    tsv_writer.writerow(['farfalla', 'rosa', 'bn:00014271n','bn:00068283n','Insetto ,grandi ali ,Lepidotteri','rosaio,Fiore,pianta'])
    tsv_writer.writerow(['recinto', 'salto', 'bn:00034048n','bn:00048558n','recinzione,barriera,cinta','movimento,salto,abilita'])
    tsv_writer.writerow(['nichilismo', 'film', 'bn:00057710n','bn:00034471n','nihilismo,Dottrina filosofica ,societa ','spettacolo ,pellicola,opera cinematografica'])
    tsv_writer.writerow(['asteroide', 'stella', 'bn:00006608n','bn:00073964n','planetoide,corpo celeste,pianeta terrestre','stella,sole,Corpo celeste'])
    tsv_writer.writerow(['sommossa', 'disegno', 'bn:00047003n','bn:00028639n','ribellione,rivolta,insurrezione','figura,illustrazione ,riproduzione'])
    tsv_writer.writerow(['intimo', 'corpo', 'bn:00079003n','bn:00011744n','biancheria intima,lingerie,indumento','organismo,tessuti,tessuti'])
    tsv_writer.writerow(['Boeing', 'aereo', 'bn:01156257n','bn:00002275n','aereo militare,mezzo militare,spazioplano','aeromobile,velivolo,aeroplano'])
    tsv_writer.writerow(['cameo', 'interpretazione', 'bn:02119437n','bn:00061560n','apparizione,personaggio famoso,spettacolo','performance,esecuzione,esibizione'])
    tsv_writer.writerow(['semestre', 'quadrimestre', 'bn:00000561n','bn:15658281n','suddivisione anno, sei mesi ,scolastico','suddivisione anno, quattro mesi,scolastico'])
    tsv_writer.writerow(['arancia', 'agrume', 'bn:17389700n','bn:00019301n','arancio ,albero,Rutaceae','Citrus,frutto,pianta'])
    tsv_writer.writerow(['ghiacciaio', 'riscaldamento globale', 'bn:00040579n','bn:00040681n','ghiaccio,regioni montane,neve','clima,atmosfera,mutamento'])
    tsv_writer.writerow(['galleria', 'percorso', 'bn:00078606n','bn:00067975n','tunnel,Passaggio,cunicolo','strada,via,Striscia di terreno'])

CONSEGNA 1: Consiste nell’annotare con punteggio di semantic similarity 50 coppie di termini.

In [43]:
def main():
    human_similarities_dictionary = get_human_similarities_dictionary()
    senses_dictionary = get_senses_dictionary(get_word_list(human_similarities_dictionary))
    NASARI_similarities_dictionary = get_NASARI_similarities_dictionary(human_similarities_dictionary, senses_dictionary)
    
    human_similarities = []
    NASARI_similarities = []
    for word_pair in human_similarities_dictionary.keys():
        if word_pair in NASARI_similarities_dictionary.keys():
            human_similarities.append(float(human_similarities_dictionary[word_pair]))
            NASARI_similarities.append(NASARI_similarities_dictionary[word_pair])
    print()
    print("VALUTAZIONI DI SIMILARITA' UMANE: ")
    print(human_similarities_dictionary)
    print()
    print("VALUTAZIONI DI SIMILARITA' DEL SISTEMA: ")
    print(NASARI_similarities_dictionary)
    print()
    print("Pearson Correlation: ",np.corrcoef(human_similarities, NASARI_similarities))
    print("Spearman Correlation: ",sp.stats.spearmanr(human_similarities, NASARI_similarities))

main()

IndexError: list index out of range

CONSEGNA 2: Consiste nell’individuare i sensi selezionati nel giudizio di similarità

In [ ]:
def main():
    human_similarities_dictionary = get_human_similarities_dictionary()
    senses_dictionary = get_senses_dictionary(get_word_list(human_similarities_dictionary))
    human_synsets_dictionary = get_human_synsets_dictionary()
    word_pair_synset_pair_dictionary = get_word_pair_synset_pair_dictionary(human_synsets_dictionary, senses_dictionary)
    
    print()
    print("ASSEGNAMENTI SYNSETS UMANI: ")
    print(human_synsets_dictionary)
    print()
    print("'ASSEGNAMENTI SYNSETS DEL SISTEMA: ")
    print(word_pair_synset_pair_dictionary)
    
    print()
    #calcolo accuratezza sui singoli elementi
    checked = 0
    for word_pair in human_synsets_dictionary.keys():
        synset_pair = word_pair_synset_pair_dictionary[word_pair]
        human_synsets_pair = human_synsets_dictionary[word_pair]
        if synset_pair[0] == human_synsets_pair[0]:
            checked += 1
        if synset_pair[1] == human_synsets_pair[1]:
            checked += 1
    evaluated = len(human_synsets_dictionary.keys()) * 2
    print("Accuratezza sui singoli elmenti: ", checked / evaluated)
    
    #calcolo accuratezza sulle coppie
    checked = 0
    for word_pair in human_synsets_dictionary.keys():
        synset_pair = word_pair_synset_pair_dictionary[word_pair]
        human_synsets_pair = human_synsets_dictionary[word_pair]
        if (synset_pair[0] == human_synsets_pair[0]) and (synset_pair[1] == human_synsets_pair[1]):
            checked += 1
    evaluated = len(human_synsets_dictionary.keys())
    print("Accuratezza sulle coppie: ", checked / evaluated)
main()